# Install


In [2]:
!pip install datasets==2.20.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.5.0 which is incompatible.


# 4.1.1 Tokenizers 학습

In [3]:
from datasets import load_dataset

dataset = load_dataset("klue", "ynat")
dataset["train"][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [4]:
target_key = "title"
for key in dataset.column_names.keys():
  with open(f"/content/tokenizer_data_{key}.txt", "w") as f:
    f.write("\n".join(dataset[key][target_key]))

In [5]:
user_defined_symbols = [
    "[PAD]",  # 문장의 길이를 맞추기 위해 사용되는 토큰
    "[UNK]",  # 토크나이저가 인식할 수 없는 토큰
    "[CLS]",  # bert 계열 모델에서 문장 전체의 정보를 저장하는 토큰
    "[SEP]",  # bert 계열 모델에서 문장 구분을 위해 사용하는 토큰
    "[MASK]", # MLM 모델에서 토큰 마스킹을 위해 사용하는 토큰
]

unused_token_num = 100
unused_list = [f"[UNUSED{i}]" for i in range(unused_token_num)]  # 사전학습 시, 어휘에 없는 토큰을 추가하기 위한 빈 공간

whole_user_defined_symbols = user_defined_symbols + unused_list
print(whole_user_defined_symbols[:10])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '[UNUSED0]', '[UNUSED1]', '[UNUSED2]', '[UNUSED3]', '[UNUSED4]']


In [6]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [7]:
from tokenizers import normalizers

normalizer = normalizers.BertNormalizer()
bert_tokenizer.normalizer = normalizer

normalizer.normalize_str("Héllò hôwWnare ü? ")

'hello howwnare u? '

In [8]:
from tokenizers.pre_tokenizers import Whitespace

pre_tokenizer = Whitespace()
bert_tokenizer.pre_tokenizer = pre_tokenizer

pre_tokenizer.pre_tokenize_str("안녕하세요. 제대로 인코딩이 되는지 확인 중입니다.")

[('안녕하세요', (0, 5)),
 ('.', (5, 6)),
 ('제대로', (7, 10)),
 ('인코딩이', (11, 15)),
 ('되는지', (16, 19)),
 ('확인', (20, 22)),
 ('중입니다', (23, 27)),
 ('.', (27, 28))]

In [9]:
from tokenizers.processors import TemplateProcessing

post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[(t, i) for i, t in enumerate(user_defined_symbols)]
)

bert_tokenizer.post_processor = post_processor

In [10]:
from tokenizers.trainers import WordPieceTrainer

vocab_size = 24000
trainer = WordPieceTrainer(
    vocab_size=vocab_size,
    special_tokens=whole_user_defined_symbols,
)

In [11]:
from glob import glob

bert_tokenizer.train(glob(f"/content/*.txt"), trainer)

In [12]:
output = bert_tokenizer.encode("인코딩 및 디코딩이 제대로 이루어지는지 확인 중입니다.")
print(output.ids)

bert_tokenizer.decode(output.ids)

[2, 675, 906, 2220, 4518, 1240, 906, 2220, 569, 6727, 12916, 10780, 586, 1881, 16617, 10191, 106, 3]


'인 ##코 ##딩 및 디 ##코 ##딩 ##이 제대로 이루 ##어지는 ##지 확인 중이 ##ᆸ니다 .'

In [13]:
from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)

'인코딩 및 디코딩이 제대로 이루어지는지 확인 중입니다.'

In [14]:
from transformers import BertTokenizerFast

fast_tokenizer = BertTokenizerFast(tokenizer_object=bert_tokenizer)
encoded = fast_tokenizer.encode("인코딩 및 디코딩이 제대로 이루어지는지 확인 중입니다.")
decoded = fast_tokenizer.decode(encoded)
print(encoded)
print(decoded)

[2, 675, 906, 2220, 4518, 1240, 906, 2220, 569, 6727, 12916, 10780, 586, 1881, 16617, 10191, 106, 3]
[CLS] 인코딩 및 디코딩이 제대로 이루어지는지 확인 중입니다. [SEP]


Save Tokenizer

In [15]:
output_dir = "/content/MyTokenizer"
fast_tokenizer.save_pretrained(output_dir)

('drive/MyDrive/MyTokenizer/tokenizer_config.json',
 'drive/MyDrive/MyTokenizer/special_tokens_map.json',
 'drive/MyDrive/MyTokenizer/vocab.txt',
 'drive/MyDrive/MyTokenizer/added_tokens.json',
 'drive/MyDrive/MyTokenizer/tokenizer.json')

In [16]:
new_tokenizer = BertTokenizerFast.from_pretrained(output_dir)

encoded = new_tokenizer(["인코딩 잘 되는지 확인", "안되면 다시 학습하자"])

for k, v in encoded.items():
  print(k, v)

print(new_tokenizer.decode(encoded["input_ids"][0]))
print(new_tokenizer.decode(encoded["input_ids"][1]))

input_ids [[2, 675, 906, 2220, 1675, 6464, 586, 1881, 3], [2, 18632, 1594, 6985, 3782, 3]]
token_type_ids [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
attention_mask [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]
[CLS] 인코딩 잘 되는지 확인 [SEP]
[CLS] 안되면 다시 학습하자 [SEP]


# 4.1.2 모델 초기화 후 학습

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizerFast

dataset = load_dataset("klue", "ynat")
model_name = "/content/MyTokenizer"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

In [ ]:
from transformers import BertConfig
cfg = BertConfig
print(cfg)

In [ ]:
mycfg = BertConfig(vocab_size=tokenizer.vocab_size)

In [ ]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(mycfg)
print(model.config)